In [6]:
cfg.DATA.PATH_TO_DATA_DIR = 'data/raw/slowfast_data_cfg/'
cfg.DATA.PATH_PREFIX = 'data/raw/video/'

In [7]:
dataloader = loader.construct_loader(cfg, 'test')

In [8]:
data_iter = iter(dataloader)
batch = data_iter.next()

In [9]:
frames, label, index = batch
device = torch.device("cuda:0")
model.eval()
frames = [x.to(device) for x in frames]
preds = model(frames)

In [10]:
x = frames[0][:2], frames[0][:2]

In [17]:
frames, label, index = batch
x = [x.to(device) for x in frames]

In [18]:
m = model
x = m.s1(x)
x = m.s1_fuse(x)
x = m.s2(x)
x = m.s2_fuse(x)
for pathway in range(m.num_pathways):
    pool = getattr(m, "pathway{}_pool".format(pathway))
    x[pathway] = pool(x[pathway])
x = m.s3(x)
x = m.s3_fuse(x)
x = m.s4(x)
x = m.s4_fuse(x)
x = m.s5(x)
# x = self.head(x)
# return x

In [19]:
# head
head = m.head
x0 = head.pathway0_avgpool(x[0])
x1 = head.pathway1_avgpool(x[1])
pool_out = [x0, x1]
h = torch.cat(pool_out, 1)
h = h.permute((0,2,3,4,1))
# h = head.dropout(h)
# h = head.projection(h)

In [20]:
x[0].shape, x[1].shape, h.shape

(torch.Size([2, 2048, 8, 8, 8]),
 torch.Size([2, 256, 32, 8, 8]),
 torch.Size([2, 1, 2, 2, 2304]))

In [22]:
h = h.mean([1,2,3])

In [ ]:
h.shape

In [3]:
11178/2600

4.299230769230769